In [0]:
import datetime as dt

try:
    arrival_date=dbutils.widgets.get("arrival_date")
except:
    arrival_date=dt.datetime.now().strftime("%Y-%m-%d")

try:
    catalog=dbutils.widgets.get('catalog')
except Exception:
    catalog="travel_bookings"

try:
    schema=dbutils.widgets.get('schema')
except Exception:
    schema="default"

try:
    base_volume=dbutils.widgest.get('base_volume')
except Exception:
    base_volume=f"/Volume/{catalog}/{schema}/data"

from pyspark.sql.functions import current_timestamp,lit,to_date
from pyspark.sql import functions as F
import time

booking_path=f"{base_volume}/booking_data/bookings_{arrival_date}.csv"
customer_path=f"{base_volume}/customer_data/customers_{arrival_date}.csv"

missing = []
try:
    dbutils.fs.ls(booking_path)
except Exception:
    missing.append(booking_path)

try:
    dbutils.fs.ls(customer_path)
except Exception:
    missing.append(customer_path)

if missing:
    raise FileNotFoundError(f"Missing input files: {missing}")



spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.ops")
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.ops.run_log (
  run_id STRING,
  arrival_date DATE,
  stage STRING,
  status STRING,
  message STRING,
  recorded_at TIMESTAMP
) USING DELTA
""")



run_id = f"nb-validate-{arrival_date}-{int(time.time())}"
log_df = spark.createDataFrame([
    (run_id, arrival_date, "validate_inputs", "STARTED", "Inputs validated")
], ["run_id","arrival_date","stage","status","message"])
log_df = log_df.withColumn("arrival_date", F.to_date("arrival_date")).withColumn("recorded_at", current_timestamp())
log_df.write.mode("append").saveAsTable(f"{catalog}.ops.run_log")

print("Validation successful:", booking_path, customer_path)


    
    